In [20]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from skimage.filters import gaussian, threshold_otsu, threshold_multiotsu, sobel, threshold_sauvola
from skimage.morphology import remove_small_objects, disk, binary_closing
from scipy.ndimage import zoom, binary_dilation, binary_erosion, distance_transform_edt
from skimage.measure import label, regionprops
from skimage import io, exposure, color
from skimage import measure, morphology
from skimage import exposure
from czifile import imread
import cv2
import re
from matplotlib.ticker import MaxNLocator
import imageio.v2 as imageio

In [21]:

def display_image(image, path, type):
    """Display the image."""
    plt.imshow(image)
    plt.axis('off')
    plt.title(f"{path} {type}")
    plt.show()

def extract_image_paths(folder):
    """Extract all image file paths from the specified folder."""
    return [os.path.join(folder, f) for f in os.listdir(folder) if os.path.isfile(os.path.join(folder, f))]

def read_image(image_path):
    """Read the LSM image from the specified path."""
    return imread(image_path)

def extract_channels(image: np.ndarray):
    """
    Extract green and red channels from an image.
    Supports both (H, W) grayscale and (C, H, W) or (H, W, C) formats.
    Always returns (green, red) where red can be None.
    """
    shape = image.shape

    # Case 1: grayscale (no channel dimension)
    if len(shape) == 2:
        return image, None

    # Case 2: (C, H, W)
    elif len(shape) == 3 and shape[0] <= 3:
        if shape[0] == 2:
            return image[0], image[1]
        else:
            return image[0], None

    # Case 3: (H, W, C)
    elif len(shape) == 3 and shape[2] <= 3:
        if shape[2] == 2:
            return image[..., 0], image[..., 1]
        else:
            return image[..., 0], None

    else:
        raise ValueError(f"Unsupported image shape: {shape}")
    
def extract_image_paths(root_folder):
    """Recursively collect all image file paths under root_folder."""
    image_paths = []
    for dirpath, _, filenames in os.walk(root_folder):
        for f in filenames:
            image_paths.append(os.path.join(dirpath, f))
    return image_paths

In [22]:
def main(image_folder):
    output_dir = "converted_images"
    os.makedirs(output_dir, exist_ok=True)

    images_to_analyze = extract_image_paths(image_folder)

    for path in images_to_analyze:
        image = read_image(path)
        image_squeezed = np.squeeze(image) 
        green, _ =  extract_channels(image_squeezed)

        # get subfolder name (relative to root)
        rel_dir = os.path.dirname(os.path.relpath(path, image_folder))
        folder_name = os.path.basename(rel_dir) or "root"

        filename = os.path.splitext(os.path.basename(path))[0]
        save_path = os.path.join(output_dir, f"{folder_name}_{filename}.png")

        # save green (or single channel)
        imageio.imwrite(save_path, green)

    print(f"✅ Done! Processed {len(images_to_analyze)} images into {output_dir}")

if __name__ == "__main__":
    image_folder = "images"
    main(image_folder)

✅ Done! Processed 337 images into converted_images
